In [13]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/dt-sc-Josue-Diaz/Proyecto-DS-II/refs/heads/main/05_models_evaluation/data_para_entrenamiento.csv')

In [15]:
df.columns

Index(['porcentaje_crediticio', 'edad', 'retrasos_bimestrales',
       'razon_de_deuda', 'ingresos', 'cantidad_de_prestamos',
       'cartera_vencida', 'deuda_inmobiliara', 'retrasos_trimestrales',
       'numero_dependientes'],
      dtype='object')

# Random Forest

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [10]:
X = df.drop(columns=['moroso'])
y = df['moroso']

KeyError: "['moroso'] not found in axis"

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [14]:
random_forest = RandomForestClassifier()

In [15]:
random_forest.fit(X_train, y_train)

RandomForestClassifier()

In [16]:
y_pred_train_rf = random_forest.predict(X_train)
y_pred_test_rf = random_forest.predict(X_test)

In [17]:
print("Training")
print(classification_report(y_train, y_pred_train_rf))
print("---------------------------------------------------------")
print("Test")
print(classification_report(y_test, y_pred_test_rf))

Training
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    102176
           1       1.00      1.00      1.00      7381

    accuracy                           1.00    109557
   macro avg       1.00      1.00      1.00    109557
weighted avg       1.00      1.00      1.00    109557

---------------------------------------------------------
Test
              precision    recall  f1-score   support

           0       0.94      0.99      0.97     34053
           1       0.53      0.19      0.27      2466

    accuracy                           0.93     36519
   macro avg       0.74      0.59      0.62     36519
weighted avg       0.92      0.93      0.92     36519



XGBOOST

In [24]:
from xgboost import XGBClassifier

In [25]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [26]:
xgboost = XGBClassifier(eval_metric='logloss')

In [27]:
xgboost.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [28]:
y_pred_train_xgb = xgboost.predict(X_train)
y_pred_test_xgb = xgboost.predict(X_test)

In [29]:
print("Training")
print(classification_report(y_train, y_pred_train_xgb))
print("---------------------------------------------------------")
print("Test")
print(classification_report(y_test, y_pred_test_xgb))

Training
              precision    recall  f1-score   support

           0       0.91      0.89      0.90     22207
           1       0.82      0.85      0.83     12793

    accuracy                           0.87     35000
   macro avg       0.86      0.87      0.86     35000
weighted avg       0.87      0.87      0.87     35000

---------------------------------------------------------
Test
              precision    recall  f1-score   support

           0       0.82      0.80      0.81      9464
           1       0.67      0.70      0.68      5536

    accuracy                           0.76     15000
   macro avg       0.74      0.75      0.74     15000
weighted avg       0.76      0.76      0.76     15000



Optimizacion: xgboost

In [30]:
from sklearn.model_selection import GridSearchCV

In [31]:
param_grid = {
    'n_estimators': [200, 300, 500],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.005],
    'subsample': [1],
    'colsample_bytree': [0.4, 0.5, 0.6],
    'min_child_weight': [1, 3, 5]
}

In [32]:
xgboost = XGBClassifier(eval_metric='logloss')

In [33]:
grid_search = GridSearchCV(estimator=xgboost, param_grid = param_grid, scoring="precision", cv=5)

In [34]:
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'colsample_bytree': [0.4, 0.5, 0.6],
                         'learning_rate': [0.01, 0.05, 0.005],
                         'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5],
                         'n_estimators': [200, 300, 500], 'subsample': [1]},
             scoring='precision')

In [35]:
best_model = grid_search.best_estimator_

In [36]:
best_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.005, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
              max_leaves=None, min_child_weight=5, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [37]:
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

In [38]:
#MODELO ANTERIOR
print("Training")
print(classification_report(y_train, y_pred_train))
print("---------------------------------------------------------")
print("Test")
print(classification_report(y_test, y_pred_test))

Training
              precision    recall  f1-score   support

           0       0.74      0.92      0.82     22207
           1       0.76      0.44      0.56     12793

    accuracy                           0.75     35000
   macro avg       0.75      0.68      0.69     35000
weighted avg       0.75      0.75      0.72     35000

---------------------------------------------------------
Test
              precision    recall  f1-score   support

           0       0.72      0.91      0.80      9464
           1       0.72      0.40      0.51      5536

    accuracy                           0.72     15000
   macro avg       0.72      0.65      0.66     15000
weighted avg       0.72      0.72      0.70     15000



In [39]:
#MODELO NUEVO
print("Training")
print(classification_report(y_train, y_pred_train))
print("---------------------------------------------------------")
print("Test")
print(classification_report(y_test, y_pred_test))

Training
              precision    recall  f1-score   support

           0       0.74      0.92      0.82     22207
           1       0.76      0.44      0.56     12793

    accuracy                           0.75     35000
   macro avg       0.75      0.68      0.69     35000
weighted avg       0.75      0.75      0.72     35000

---------------------------------------------------------
Test
              precision    recall  f1-score   support

           0       0.72      0.91      0.80      9464
           1       0.72      0.40      0.51      5536

    accuracy                           0.72     15000
   macro avg       0.72      0.65      0.66     15000
weighted avg       0.72      0.72      0.70     15000



Guardar el modelo

In [40]:
import joblib

In [ ]:
pip install joblib

In [41]:
joblib.dump(best_model, "C:/Users/tinch/coderhouse/modelo_template/models/xgboost_model.joblib")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/tinch/coderhouse/modelo_template/models/xgboost_model.joblib'

Ver distribuciones

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
probabilidades_train = best_model.predict_proba(X_train)[:,1]
probabilidades_test = best_model.predict_proba(X_train)[:,1]

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(probabilidades_train, bins=30, kde=True, color='skyblue')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(probabilidades_test, bins=30, kde=True, color='orange')
plt.show()